### About the data...

In [1]:
import logging
import config
from IPython import display
from dataset.TPL import *
import dataset.preprocessing as pre
import numpy as np
import pandas as pd
import time 
from gensim import corpora, models, similarities
%reload_ext autoreload
%autoreload 2
logging.basicConfig(level=logging.INFO)

In [2]:
config.ds_medium

{'startTimestamp': '2020-12-01T00:00:00.000',
 'stopTimestamp': '2021-01-02T00:00:00.000',
 'dataset_dir': '../../../data/raw',
 'processed_dir': '../../../data/processed'}

In [3]:
#Importamos el dataset con el cual vamos a trabajar:
pionier1 = TplPionier(config=config.ds_medium)
matisse1 = TplMatisse(config=config.ds_medium)
gravity1  = TplGravity(config=config.ds_medium)

### Utilicemos a Gravity como conejillo de indias...

In [4]:
#Mostramos el dataset asociado al gravity
gravity1.index

,START,END,TIMEOUT,system,procname,TPL_ID,ERROR,Aborted,estimated (s),real (s),diff (s),SECONDS
0,2020-12-01 00:16:04.547000+00:00,2020-12-01 00:19:11.750000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_exp,False,False,227,307,40,187.0
1,2020-12-01 00:19:11.804000+00:00,2020-12-01 00:23:49.341000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_swap,False,False,500,438,22,277.0
2,2020-12-01 00:23:49.411000+00:00,2020-12-01 00:25:34.908000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_exp,False,False,138,145,7,105.0
3,2020-12-01 00:25:34.967000+00:00,2020-12-01 00:29:33.402000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_swap,False,False,500,359,61,238.0
4,2020-12-01 00:29:33.453000+00:00,2020-12-01 00:31:18.444000+00:00,False,GRAVITY,bob_4415,GRAVITY_dual_obs_exp,False,False,138,145,7,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1245,2021-01-01 20:10:20.661000+00:00,2021-01-01 20:11:58.709000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_PTref,False,False,135,138,3,98.0
1246,2021-01-01 20:11:58.798000+00:00,2021-01-01 20:13:26.902000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_WFSref,False,False,123,128,5,88.0
1247,2021-01-01 20:13:26.983000+00:00,2021-01-01 20:14:47.953000+00:00,False,GRAVITY,bob_28479,GRAVITY_gen_tec_SaveDb,False,False,120,121,1,80.0
1248,2021-01-01 23:51:11.275000+00:00,2021-01-01 23:51:11.967000+00:00,False,GRAVITY,bob_28479,GRAVITY_dual_acq_dither,False,False,1000,0,600,0.0


In [5]:
##Visualizamos columnas en específico y le agregamos el número de eventos a cada template.
gravity1_index = gravity1.index[['START', 'system', 'ERROR', 'SECONDS']]
gravity1_index['# of events'] = gravity1_index.index.map( lambda i: len(gravity1.load_trace(i)) )
gravity1_index

,START,system,ERROR,SECONDS,# of events
0,2020-12-01 00:16:04.547000+00:00,GRAVITY,False,187.0,1795
1,2020-12-01 00:19:11.804000+00:00,GRAVITY,False,277.0,1408
2,2020-12-01 00:23:49.411000+00:00,GRAVITY,False,105.0,1146
3,2020-12-01 00:25:34.967000+00:00,GRAVITY,False,238.0,1247
4,2020-12-01 00:29:33.453000+00:00,GRAVITY,False,104.0,1124
...,...,...,...,...,...
1245,2021-01-01 20:10:20.661000+00:00,GRAVITY,False,98.0,982
1246,2021-01-01 20:11:58.798000+00:00,GRAVITY,False,88.0,870
1247,2021-01-01 20:13:26.983000+00:00,GRAVITY,False,80.0,663
1248,2021-01-01 23:51:11.275000+00:00,GRAVITY,False,0.0,69


In [6]:
#Visualizamos los eventos asociados a la trace(0)
gravity1.load_trace(0)["event"]

0             Started at 2020-12-01T00:16:04 (underlined)
1          GRAVITY_dual_obs_exp -- Dual Field Science ...
2                                    SEQ RELOFF Y = '0.0'
3                                    SEQ OBSSEQ = 'O S O'
4                                   INS SPEC RES = 'SAME'
                              ...                        
1790                     'finally' block executed (brown)
1791                       ###################### (brown)
1792        Template Dual Field Science finished. (brown)
1793       Finished in 187 seconds at 2020-12-01T00:19...
1794       TplExecTimeStats: TPL.ID GRAVITY_dual_obs_e...
Name: event, Length: 1795, dtype: object

Bien, ahora que ya tuvimos nuestro primer acercamiento con la información que vamos a trabajar, corresponde dar el siguiente paso, pero primero importamos la función que nos ayudará a la limpieza de información:

In [7]:
## preprocessing
class NumbersAndTelescopes(pre.Numbers, pre.Telescopes):
    pass

colorize1 = pre.Numbers()
colorize2 = NumbersAndTelescopes()

In [8]:
print(colorize2.help())
print('\n---------')

Colorization class "NumbersAndTelescopes"
Inherits from: Numbers, Telescopes, BaseColor

Colorization is done in the following order:

Numbers: Transform numbers in tokens
    -10.54 -> {}
    9.1e-2.1 -> {}
Telescopes: Tokenize telescope related hardware
    wat1tcs -> {watXtcs}
    lat2trk -> {latXtrk}

---------


### Proceso de creación del corpus:

La primera idea fue la siguiente: juntar todos los eventos de cada template y guardarlos en una misma variable, con tal de hacer andar el modelo y practicar para la creación del corpus y LDA

In [10]:
## Juntamos todos los event de los trace's en traces, contamos con 3377953 events 
traces = gravity1.load_trace(0, cols=['@timestamp', 'event'])
for i in range(1,len(gravity1.index)): 
       traces=traces.append(gravity1.load_trace(i, cols=['@timestamp', 'event']))

In [11]:
##Mostramos los 3377953 events para comprobar que todo anda bien
with pd.option_context('display.max_colwidth', None):
    display( traces )

,@timestamp,event
0,2020-12-01 00:16:04.547000+00:00,Started at 2020-12-01T00:16:04 (underlined)
1,2020-12-01 00:16:04.547000+00:00,GRAVITY_dual_obs_exp -- Dual Field Science (yellow)
2,2020-12-01 00:16:04.770000+00:00,SEQ RELOFF Y = '0.0'
3,2020-12-01 00:16:04.770000+00:00,SEQ OBSSEQ = 'O S O'
4,2020-12-01 00:16:04.770000+00:00,INS SPEC RES = 'SAME'
...,...,...
67,2021-01-01 23:51:26.813000+00:00,hour 0 min 0 sec 0.000
68,2021-01-01 23:51:26.815000+00:00,hour 0 min 0 sec 0.000
69,2021-01-01 23:51:26.816000+00:00,Finished in 0 seconds at 2021-01-01T23:51:26 (underlined)
70,2021-01-01 23:51:26.816000+00:00,"expected floating-point number but got ""UNDEF"" (red)"


In [13]:
#Agregamos una nueva columna a nuestro "trace" y le aplicamos funciones de limpieza
traces['color2'] = traces['event'].apply(NumbersAndTelescopes().color)
traces.describe()

,@timestamp,event,color2
count,3377953,3377953,3377953
unique,1252312,374211,21980
top,2020-12-26 14:36:28.622000+00:00,INS.LAMP19.START Lamp turned on.,INS.LAMP19.START Lamp turned on.
freq,90,130293,130293
first,2020-12-01 00:16:04.547000+00:00,NaN,NaN
last,2021-01-01 23:51:26.821000+00:00,NaN,NaN


In [14]:
##Visualizamos la comparación de los primeros 20 event y sus respectivos tokens
traces

,@timestamp,event,color2
0,2020-12-01 00:16:04.547000+00:00,Started at 2020-12-01T00:16:04 (underlined),Started at {} (underlined)
1,2020-12-01 00:16:04.547000+00:00,GRAVITY_dual_obs_exp -- Dual Field Science ...,GRAVITY_dual_obs_exp -- Dual Field Science (ye...
2,2020-12-01 00:16:04.770000+00:00,SEQ RELOFF Y = '0.0',SEQ RELOFF Y = '{}'
3,2020-12-01 00:16:04.770000+00:00,SEQ OBSSEQ = 'O S O',SEQ OBSSEQ = 'O S O'
4,2020-12-01 00:16:04.770000+00:00,INS SPEC RES = 'SAME',INS SPEC RES = 'SAME'
...,...,...,...
67,2021-01-01 23:51:26.813000+00:00,hour 0 min 0 sec 0.000,hour {} min {} sec {}
68,2021-01-01 23:51:26.815000+00:00,hour 0 min 0 sec 0.000,hour {} min {} sec {}
69,2021-01-01 23:51:26.816000+00:00,Finished in 0 seconds at 2021-01-01T23:51:2...,Finished in {} seconds at {} (underlined)
70,2021-01-01 23:51:26.816000+00:00,"expected floating-point number but got ""UND...","expected floating-point number but got ""UNDEF""..."


El problema fue el siguiente, cuento con una lista que contiene todos los tokens de mi dataset en orden de aparición, pero...¿realmente estoy trabajando con el formato adecuado? revisando otros ejemplos puedo afirmar que no, por lo que intentemos respetar este formato de "fila(i) -> tokens asociados a fila (i)"

In [21]:
#Creamos una lista que contenga los eventos de la primera trace colorizados:
first_trace=[]
for a in gravity1.load_trace(0)["event"]:
    first_trace.append(colorize2.color(a))
len(first_trace)

1795

In [23]:
#La idea acá no es nada más que probar el formato para efectos de un número pequeño de eventos y ver si funciona, 
#todo muy manual
second_trace=[]
for a in gravity1.load_trace(1)["event"]:
    g.append(colorize2.color(a))

In [26]:
#nos apoyamos de pandas
serie = pd.Series(([first_trace,second_trace]))
serie

0    [Started at {} (underlined), GRAVITY_dual_obs_...
1    [Started at {} (underlined), GRAVITY_dual_obs_...
dtype: object

¡Construimos lo que necesitamos! pero ahora llevémoslo a función:

In [27]:
def generate_serie(obs):
    
    #obs: dataset
    firstrow=[]
    rows=[]
    
    #we create the first row for our serie:   
    for a in obs.load_trace(0)["event"]:
        firstrow.append(colorize2.color(a))
        
    #and then we create the serie:
    serie= pd.Series([firstrow])
    
    #now for the rest of traces
    for i in range(1, len(obs.index)):
    
        for a in obs.load_trace(i)["event"]:
            rows.append(colorize2.color(a))
                
        serie=serie.append(pd.Series([rows],index=[i]))
        rows=[]
    
    return serie

Veamos qué tal:

In [29]:
pionier1_data=generate_serie(pionier1)
matisse1_data=generate_serie(matisse1)
gravity1_data=generate_serie(gravity1)

In [39]:
pionier1_data

0       [Started at {} (underlined), INS MODE = 'OBSER...
1       [Started at {} (underlined), PIONIER_gen_tec_s...
2       [PIONIER_gen_tec_scan -- Scanning mode setup: ...
3       [Started at {} (underlined), PIONIER_gen_tec_s...
4       [Started at {} (underlined), DPR CATG = 'CALIB...
                              ...                        
2841    [PIONIER_gen_tec_setup -- Instrument setup (FR...
2842    [Started at {} (underlined), PIONIER_gen_tec_s...
2843    [PIONIER_gen_tec_setup -- Instrument setup (ye...
2844    [Started at {} (underlined), DET NDIT = '{}', ...
2845    [Started at {} (underlined), PIONIER_gen_tec_s...
Length: 2846, dtype: object

In [40]:
matisse1_data

0       [Started at {} (underlined), MATISSE_gen_tec_a...
1       [MATISSE_gen_tec_ali -- Align L-Band (yellow),...
2       [Started at {} (underlined), SEQ ALIGN MODE = ...
3       [Started at {} (underlined), MATISSE_gen_tec_a...
4       [Started at {} (underlined), MATISSE_gen_tec_a...
                              ...                        
1311    [MATISSE_gen_tec_ali -- Align L-Band (yellow),...
1312    [MATISSE_gen_tec_opd -- Rough cophasing CPL (y...
1313    [Started at {} (underlined), MATISSE_gen_tec_o...
1314    [Started at {} (underlined), MATISSE_gen_tec_o...
1315    [MATISSE_gen_tec_opd -- Fine cophasing BCOP (y...
Length: 1316, dtype: object

In [41]:
gravity1_data

0       [Started at {} (underlined), GRAVITY_dual_obs_...
1       [Started at {} (underlined), GRAVITY_dual_obs_...
2       [GRAVITY_dual_obs_exp -- Dual Field Science (y...
3       [Started at {} (underlined), GRAVITY_dual_obs_...
4       [GRAVITY_dual_obs_exp -- Dual Field Science (y...
                              ...                        
1245    [Started at {} (underlined), GRAVITY_gen_tec_P...
1246    [Started at {} (underlined), GRAVITY_gen_tec_W...
1247    [GRAVITY_gen_tec_SaveDb -- Save Instrument Dat...
1248    [GRAVITY_dual_acq_dither -- Dithering {} (yell...
1249    [GRAVITY_dual_acq_dither -- Dithering {} (yell...
Length: 1250, dtype: object

Finalmente obtuvimos el formato deseado para poder comenzar a trabajar.